In [2]:
import findspark
findspark.init()
import pyspark
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import DataFrameReader, SQLContext, SparkSession

try:
    sc.stop() 
except:
    pass
sc = pyspark.SparkContext(master='spark://172.17.0.3:7077', appName='app15')


# PEC1

![pokemon](https://kaggle2.blob.core.windows.net/datasets-images/2619/4359/e3ef5846d64dc9a747afd82273456328/dataset-cover.jpg)

# Ejercicio 1: Operaciones básicas
## 1.1 Lectura de datos CSV y renombrado de columnas
Lee en un DataFrame el contenido de los datos que están en el CSV con la siguiente ruta: `data/pokemon/pokemon.csv`

Renombra las columnas del dataframe para que sean las siguientes:

`"id", "name", "type_1", "type_2", "hp", "attack", "defense", "special_attack", "special_defense", "speed", "generation", "legendary"`

In [3]:
### Incluye aquí tu respuesta ###
sqlContext = pyspark.SQLContext(sc)


df = sqlContext.read.format("csv").load("hdfs://172.17.0.9:8020/ejemplo1/pokemon.csv", header=True).toDF("id", "name", "type_1", "type_2", "hp", "attack", "defense", "special_attack", "special_defense", "speed", "generation", "legendary")
df.createOrReplaceTempView("pokemon")

## 1.2 Comprobación de lectura y esquema
¿Cómo puedes comprobar el esquema que ha inferido Spark al leer el CSV?


In [4]:
### Incluye aquí tu respuesta ###
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type_1: string (nullable = true)
 |-- type_2: string (nullable = true)
 |-- hp: string (nullable = true)
 |-- attack: string (nullable = true)
 |-- defense: string (nullable = true)
 |-- special_attack: string (nullable = true)
 |-- special_defense: string (nullable = true)
 |-- speed: string (nullable = true)
 |-- generation: string (nullable = true)
 |-- legendary: string (nullable = true)



En algunas ocasiones, al leer un CSV, que no se "parsea" correctamente, y las columnas pueden contener contenido parcial ó erróneo.

¿Cómo puedes comprobar que la lectura de los datos se ha realizado bien y que los campos no están "descolocados"? 

Fíjate sólo en la primera fila del DataFrame para ello

In [5]:
### Incluye aquí tu respuesta ###
df.show(1)

+---+---------+------+------+---+------+-------+--------------+---------------+-----+----------+---------+
| id|     name|type_1|type_2| hp|attack|defense|special_attack|special_defense|speed|generation|legendary|
+---+---------+------+------+---+------+-------+--------------+---------------+-----+----------+---------+
|  1|Bulbasaur| Grass|Poison| 45|    49|     49|            65|             65|   45|         1|    False|
+---+---------+------+------+---+------+-------+--------------+---------------+-----+----------+---------+
only showing top 1 row



¿Crees que Spark ha inferido bien el esquema?

In [7]:
### Incluye aquí tu respuesta ###
#Si, pero no tiene los tipo de datos correctos

¿Cambiarías el tipo de alguna columna? ¿En caso afirmativo, cómo?

In [8]:
### Incluye aquí tu respuesta ###
df2 = df.select(df['name'].cast('string')
                , df['type_1'].cast('string')
                , df['type_2'].cast('string')
                , df['hp'].cast('integer')
                , df['attack'].cast('integer')
                , df['defense'].cast('integer')
                , df['special_attack'].cast('integer')
                , df['special_defense'].cast('integer')
                , df['speed'].cast('integer')
                , df['generation'].cast('integer')
                , df['legendary'].cast('boolean')) 
df2.printSchema()

root
 |-- name: string (nullable = true)
 |-- type_1: string (nullable = true)
 |-- type_2: string (nullable = true)
 |-- hp: integer (nullable = true)
 |-- attack: integer (nullable = true)
 |-- defense: integer (nullable = true)
 |-- special_attack: integer (nullable = true)
 |-- special_defense: integer (nullable = true)
 |-- speed: integer (nullable = true)
 |-- generation: integer (nullable = true)
 |-- legendary: boolean (nullable = true)



## 1.3 Transformaciones sobre el DataFrame
Realiza las siguientes operaciones sobre el DataFrame:

### 1.3.1 Filtra las filas que no tengan un mínimo de 50 de `hp`

In [9]:
### Incluye aquí tu respuesta ###
sqlContext.sql("select * from pokemon where hp>50").count()

589

### 1.3.2 Agrupa todas las filas por el valor de la columna `type_1`

In [10]:
### Incluye aquí tu respuesta ###
sqlContext.sql("select first(type_1) as type_1 from pokemon group by type_1").show()

+--------+
|  type_1|
+--------+
|   Water|
|  Poison|
|   Steel|
|    Rock|
|     Ice|
|   Ghost|
|   Fairy|
| Psychic|
|  Dragon|
|  Flying|
|     Bug|
|Electric|
|    Fire|
|  Ground|
|    Dark|
|Fighting|
|   Grass|
|  Normal|
+--------+



### 1.3.3 Obtén el tamaño de cada grupo (es decir, cuántos pokemon de cada grupo hay)

In [11]:
### Incluye aquí tu respuesta ###
sqlContext.sql("select first(type_1) as type_1,count(type_1) as cant from pokemon group by type_1" ).show()

+--------+----+
|  type_1|cant|
+--------+----+
|   Water| 112|
|  Poison|  28|
|   Steel|  27|
|    Rock|  44|
|     Ice|  24|
|   Ghost|  32|
|   Fairy|  17|
| Psychic|  57|
|  Dragon|  32|
|  Flying|   4|
|     Bug|  69|
|Electric|  44|
|    Fire|  52|
|  Ground|  32|
|    Dark|  31|
|Fighting|  27|
|   Grass|  70|
|  Normal|  98|
+--------+----+



### 1.3.4 Quédate sólo con los 10 grupos más grandes, ordenados de mayor a menor

In [12]:
### Incluye aquí tu respuesta ###
sqlContext.sql("select first(type_1) as type_1,count(type_1) as cant from pokemon group by type_1 order by cant desc limit 10").show()

+--------+----+
|  type_1|cant|
+--------+----+
|   Water| 112|
|  Normal|  98|
|   Grass|  70|
|     Bug|  69|
| Psychic|  57|
|    Fire|  52|
|Electric|  44|
|    Rock|  44|
|  Ground|  32|
|   Ghost|  32|
+--------+----+



### 1.3.5 Trae a memoria, como un array de Rows, el resultado (los 10 grupos más grandes)

In [1]:
### Incluye aquí tu respuesta ###
arreglo = sqlContext.sql("select first(type_1) as type_1,count(type_1) as cant from pokemon group by type_1 order by cant desc limit 10")

NameError: name 'sqlContext' is not defined

### 1.3.6 De todos los anteriores pasos de este apartado 1.3 ¿cuáles son transformaciones y cuáles acciones? ¿por qué?

In [14]:
### Incluye aquí tu respuesta ###

# Ejercicio 2: Más operaciones sobre DataFrames

## 2.1 Contando elementos distintos
Queremos saber cuántos grupos distintos de pokemon **del conjunto de datos original** existen teniendo en cuenta sus dos tipos (`type_1` y `type_2`).

Por ejemplo, si nuestro conjunto de datos fuera éste:

|id|type_1|type_2|
---|-------|-----|
|1  |Water  |Grass|
|2  |Fire   |Rock |
|3  |Fire   |     |
|4  |Water  |Grass|

Entonces, la respuesta sería **3** (hay 3 grupos diferentes: `(Water, Grass)`,`(Fire, Rock)` y`(Fire, null)`

¿Cómo sacarías el número de grupos diferentes?


In [15]:
### Incluye aquí tu respuesta ###
sqlContext.sql("select type_1,type_2 from pokemon group by type_1,type_2").count()

154

## 2.2 Muestra Aleatoria
¿Cómo extraerías una muestra aleatoria de un 15% **del conjunto de datos original**?

¿Qué tamaño esperarías que debería tener esa muestra?

¿Qué tamaño ha resultado tener después de calcularla?

¿Porqué crees que no es exactamente el tamaño esperado?

In [16]:
### Incluye aquí tus respuestas ###
seed = 11
withReplacement = False
fraction = 0.15
df2.sample(withReplacement, fraction, seed).count()

134

## 2.3 Reparticionado y escritura a disco
¿Cuántas particiones tiene tu DataFrame?


In [17]:
### Incluye aquí tu respuesta ###
df2.rdd.getNumPartitions()

1

Escribe tu DataFrame, en formato CSV, en la siguiente ruta: `data/output/pokemon_default`


In [20]:
### Incluye aquí tu respuesta ###
#Note que el nuevos archivos csv se almacena en un hadoop cluster 
df2.write.csv("hdfs://172.17.0.9:8020/ejemplo1/pokemon_default.csv", header=True)

¿cuántos ficheros se han generado? ¿Por qué crees que se ha generado ese número de ficheros?

In [ ]:
### Incluye aquí tu respuesta ###
#solo uno porque solo hay un partición de datos

¿Cómo harías para escribir 4 ficheros de salida en la ruta `data/output/four_files`?

In [23]:
### Incluye aquí tu respuesta ###
repartitioned = df2.repartition(4)

In [24]:
repartitioned.rdd.getNumPartitions()

4

In [22]:
#repartitioned.write.csv("data/output/pokemon_default/four_files/mi_dataset_4.csv")
#Note que el nuevos archivos csv se almacena en un hadoop cluster 
df2.repartition(4).write.csv("hdfs://172.17.0.9:8020/ejemplo1/pokemon_default_4.csv", header=True)

# Ejercicio 3: Creando nuevas columnas

## 3.1 Nueva columna booleana
Crea una nueva columna, que se llame `strong`, de tipo booleano, que indique si un pokemon tiene más de 75 en `hp` y en alguna de las columnas `attack` ó `defense`.
Es decir, si nuestro dataset fuera:

|id|hp|attack|defense|
---|--|-----|-----|
|1  |50|90  |90|
|2  |80|35   |78 |
|3  |60|30   | 75|
|4  |90|60  |60|

El resultado esperado sería:


|id|hp|attack|defense|strong|
---|--|-----|-----|---------|
|1  |50|90  |90| false|
|2  |80|35   |78 | true|
|3  |60|30   | 75| false|
|4  |90|60  |60| false|


In [30]:
### Incluye aquí tu respuesta ###
from pyspark.sql.functions import col
df2.withColumn("strong",col("hp")>75).where("attack>75 OR defense>74").select("hp","attack","defense","strong").show()

+---+------+-------+------+
| hp|attack|defense|strong|
+---+------+-------+------+
| 80|    82|     83|  true|
| 80|   100|    123|  true|
| 78|    84|     78|  true|
| 78|   130|    111|  true|
| 78|   104|     78|  true|
| 59|    63|     80| false|
| 79|    83|    100|  true|
| 79|   103|    120|  true|
| 65|    90|     40| false|
| 65|   150|     40| false|
| 83|    80|     75|  true|
| 83|    80|     80|  true|
| 55|    81|     60| false|
| 65|    90|     65| false|
| 60|    85|     69| false|
| 60|    90|     55| false|
| 50|    75|     85| false|
| 75|   100|    110| false|
| 90|    92|     87|  true|
| 81|   102|     77|  true|
+---+------+-------+------+
only showing top 20 rows



## 3.2 Nueva columna numérica
Nos hemos inventado una nueva cualidad de los pokemon, `stamina` que responde a la fórmula:

\begin{align}
\ \frac{(hp^2 * attack)}{speed * defense}
\end{align}

Crea una nueva columna `stamina` con el valor para cada pokemon

In [31]:
### Incluye aquí tu respuesta ###
from pyspark.sql.functions import pow
df2.withColumn("stamina",(pow(col("hp"),2)*col("attack"))/(col("speed")*col("defense"))).\
    select("name","hp","attack","speed","defense","stamina").show()

+----------------+---+------+-----+-------+------------------+
|            name| hp|attack|speed|defense|           stamina|
+----------------+---+------+-----+-------+------------------+
|       Bulbasaur| 45|    49|   45|     49|              45.0|
|         Ivysaur| 60|    62|   60|     63| 59.04761904761905|
|        Venusaur| 80|    82|   80|     83| 79.03614457831326|
|   Mega Venusaur| 80|   100|   80|    123| 65.04065040650407|
|      Charmander| 39|    52|   65|     43| 28.29767441860465|
|      Charmeleon| 58|    64|   80|     58|              46.4|
|       Charizard| 78|    84|  100|     78|             65.52|
|Mega Charizard X| 78|   130|  100|    111| 71.25405405405405|
|Mega Charizard Y| 78|   104|  100|     78|             81.12|
|        Squirtle| 44|    48|   43|     65|  33.2479427549195|
|       Wartortle| 59|    63|   58|     80|  47.2635775862069|
|       Blastoise| 79|    83|   78|    100| 66.41064102564103|
|  Mega Blastoise| 79|   103|   78|    120| 68.67767094

## 3.3 Nueva columna a partir de extracción sobre strings
Crea una nueva columna que tenga como valor "Mega" si el nombre del pokemon contiene la palabra "Mega" o null en otro caso.
Pista: utiliza una expresión regular para hacer una extracción sobre la columna `name`

In [45]:
### Incluye aquí tu respuesta ###
from pyspark.sql import functions as F
df2.withColumn("Mega",F.when(col("name").like("%Mega%"),"Mega").otherwise(None)).select("name","Mega").show()

+----------------+----+
|            name|Mega|
+----------------+----+
|       Bulbasaur|null|
|         Ivysaur|null|
|        Venusaur|null|
|   Mega Venusaur|Mega|
|      Charmander|null|
|      Charmeleon|null|
|       Charizard|null|
|Mega Charizard X|Mega|
|Mega Charizard Y|Mega|
|        Squirtle|null|
|       Wartortle|null|
|       Blastoise|null|
|  Mega Blastoise|Mega|
|        Caterpie|null|
|         Metapod|null|
|      Butterfree|null|
|          Weedle|null|
|          Kakuna|null|
|        Beedrill|null|
|   Mega Beedrill|Mega|
+----------------+----+
only showing top 20 rows



## 3.4 Nueva Columna de tipo complejo
Por último, queremos crear una nueva columna `properties`,  que sea un `Map` que contenga las propiedades  más importantes de cada pokemon.
En concreto, queremos que tenga las siguientes claves y valores:

```python
{ name -> "name",
  type -> "type_1",
  hp -> hp,
  attack -> attack,
  defense -> defense
}```

¿Cómo crearías esta nueva columna?

In [49]:
### Incluye aquí tu respuesta ###
from pyspark.sql.functions import create_map,lit
df2.withColumn("properties", 
               create_map(lit("name"),col("name")
                          ,lit("type"),col("type_1")
                          ,lit("hp"),col("hp")
                          ,lit("attack"),col("attack")
                          ,lit("defense"),col("defense"))
              ).select("name","properties").show(truncate=False)

+----------------+------------------------------------------------------------------------------------+
|name            |properties                                                                          |
+----------------+------------------------------------------------------------------------------------+
|Bulbasaur       |Map(defense -> 49, name -> Bulbasaur, attack -> 49, hp -> 45, type -> Grass)        |
|Ivysaur         |Map(defense -> 63, name -> Ivysaur, attack -> 62, hp -> 60, type -> Grass)          |
|Venusaur        |Map(defense -> 83, name -> Venusaur, attack -> 82, hp -> 80, type -> Grass)         |
|Mega Venusaur   |Map(defense -> 123, name -> Mega Venusaur, attack -> 100, hp -> 80, type -> Grass)  |
|Charmander      |Map(defense -> 43, name -> Charmander, attack -> 52, hp -> 39, type -> Fire)        |
|Charmeleon      |Map(defense -> 58, name -> Charmeleon, attack -> 64, hp -> 58, type -> Fire)        |
|Charizard       |Map(defense -> 78, name -> Charizard, attack -